In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

In [62]:
# system size (2D, L*L square lattice)
L = 3
N = 4
U = 0         # take tunneling strength as a unit
sites = L**2
H1 = np.zeros((sites,sites))
for i in range(sites-1):
    H1[i,i+1] = -1
    H1[i,(i+L)%sites] = -1
for j in range(1,L):
    H1[j*L-1, j*L] = 0
    H1[j*L-1, (j-1)*L] = -1
H1[sites-1, L-1] = -1
H1[sites-1, sites-L] = -1
H1 = H1 + H1.T
H = np.kron(np.identity(2), H1)
e, phi = np.linalg.eigh(H)
phi_0 = phi[: , np.argsort(e)[:N]]
print(e)
print(phi_0)
print(np.sum(np.sort(e)[:N]))

[-4. -4. -1. -1. -1. -1. -1. -1. -1. -1.  2.  2.  2.  2.  2.  2.  2.  2.]
[[ 0.         -0.33333333 -0.66220582  0.0698694 ]
 [ 0.         -0.33333333 -0.23195751 -0.38240743]
 [ 0.         -0.33333333 -0.21201088 -0.54668478]
 [ 0.         -0.33333333 -0.10849202  0.55603114]
 [ 0.         -0.33333333  0.32175629  0.10375431]
 [ 0.         -0.33333333  0.34170292 -0.06052304]
 [ 0.         -0.33333333 -0.10974541  0.44293047]
 [ 0.         -0.33333333  0.3205029  -0.00934636]
 [ 0.         -0.33333333  0.34044953 -0.17362371]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]
 [-0.33333333  0.          0.          0.        ]]
-10.000

In [71]:
v = 0
x_up = np.random.choice(sites, np.int(N/2)+1, replace=False)
x_down = np.random.choice(sites, np.int(N/2)-1, replace=False)  # 注意要和所选取的轨道的自旋匹配，否则交叠积分为零
x = np.append(x_up, x_down+sites)
site_empty = np.delete(np.arange(2*sites), x)
site_double = np.intersect1d(x_up, x_down)
site_single = np.setdiff1d(x, site_double)
site_double_number = np.size(site_double)
print(x, x_up, site_empty, site_double, site_double_number)
print(phi_0[x])

[5 3 7 9] [5 3 7] [ 0  1  2  4  6  8 10 11 12 13 14 15 16 17] [] 0
[[ 0.         -0.33333333  0.34170292 -0.06052304]
 [ 0.         -0.33333333 -0.10849202  0.55603114]
 [ 0.         -0.33333333  0.3205029  -0.00934636]
 [-0.33333333  0.          0.          0.        ]]


In [72]:
# energy and variance
E = 0
sigma = 0

U = 0       # for test

In [73]:
# 输出与input仅相差一个hopping的configuration
def onehop(x_in):
    result_hop = []

    # left hop
    for i in range(len(x_in)):
        if x_in[i] % L == 0:
            if x_in[i]+L-1 in x_in:
                continue
            else:
                left = x_in.copy()
                left[i] = x_in[i]+L-1
                result_hop.append(left)    

        elif x_in[i]-1 in x_in:
            continue

        else:
            left = x_in.copy()
            left[i] = left[i] - 1
            result_hop.append(left)


    # right hop
    for i in range(len(x_in)):
        if x_in[i] % L == L-1:
            if x_in[i]+1-L in x_in:
                continue
            else:
                right = x_in.copy()
                right[i] = x_in[i]+1-L
                result_hop.append(right)

        elif x_in[i]+1 in x_in:
            continue

        else:
            right = x_in.copy()
            right[i] = right[i] + 1
            result_hop.append(right)
        

    # up hop
    for i in range(len(x_in)):
        if x_in[i] in range(L):
            if x_in[i]-L+sites in x_in:
                continue
            else:
                up = x_in.copy()
                up[i] = x_in[i]-L+sites
                result_hop.append(up)    

        elif x_in[i]-L in x_in:
            continue

        else:
            up = x_in.copy()
            up[i] = up[i] - L
            result_hop.append(up)

    # down hop
    for i in range(len(x_in)):
        if sites-1-x_in[i] in range(L):
            if x_in[i]+L-sites in x_in:
                continue
            else:
                down = x_in.copy()
                down[i] = x_in[i]+L-sites
                result_hop.append(down)    

        elif x_in[i]+L in x_in:
            continue

        else:
            down = x_in.copy()
            down[i] = down[i] + L
            result_hop.append(down)

    return result_hop
    

## Computation of expectations and variances of local operators $O$ (here for Hamiltonian $H$ especially)

$$ \langle O \rangle = \frac{\langle \Psi_J |O| \Psi_J \rangle}{\langle \Psi_J | \Psi_J \rangle} = \frac{ \sum_x |\langle \Psi_J| x\rangle |^2 \frac{\langle x |O| \Psi_J \rangle}{\langle x| \Psi_J \rangle}}{\sum_x |\langle \Psi_J| x\rangle |^2} = \sum_x p_J(x) O_L(x) $$

$$ H_{local} = \frac{\langle x |H| \Psi_J \rangle}{\langle x| \Psi_J \rangle} = \frac{ \sum_{x^{\prime}} \langle x |T| x^{\prime} \rangle \Psi_J(x^{\prime}) + U(x)\Psi_J(x)}{\Psi_J(x)} $$

$$ H^2_{local} = \frac{\langle x |(U+T)^2| \Psi_J \rangle}{\langle x| \Psi_J \rangle}  = \frac{ \sum_{x^{\prime \prime}} \langle x |T^2| x^{\prime \prime} \rangle \langle x^{\prime \prime}| \Psi_J \rangle + U(x) \langle x |T| \Psi_J \rangle + \sum_{x^{\prime}}\langle x |T| x^{\prime} \rangle U(x^{\prime}) \Psi_J(x^{\prime})+ U^2(x)\Psi_J(x)}{\ \Psi_J(x)} $$

$| x^{\prime} \rangle $ and $| x^{\prime \prime} \rangle $ are configurations differ from $| x \rangle $ with only one and two hopping and no change in spin.

So there are three terms about $| x^{\prime} \rangle $ and one term about $| x^{\prime \prime} \rangle $

In [ ]:
# 对于输入的configuration x 给出局域的能量H_local(x)和方差（定义见上式），示例： energy, var = energy_var(x_up_in, x_down_in)
def energy_var(x_up_in, x_down_in):
    x_in = np.append(x_up_in, x_down_in+sites)
    site_double = np.intersect1d(x_up_in, x_down_in)
    double = np.size(site_double)

    psi_x = np.linalg.det(phi_0[x_in])

    jx = np.exp(-v * double)
    psi_jx = psi_x * jx
    ux = U * double

    onehop_up = onehop(x_up_in)
    onehop_down = onehop(x_down_in)
    twohop_up = []
    for item in onehop_up:
        twohop_up = twohop_up + onehop(item)
    twohop_down = []
    for item in onehop_down:
        twohop_down = twohop_down + onehop(item)

    # 只跳一次的话由以下两种情况构成：up一次down零次， up零次down一次。下面的式子按此排序
    x_prime_up = onehop_up.copy()
    for i in range(len(onehop_down)):
        x_prime_up.append(x_up_in)

    x_prime_down = []
    for i in range(len(onehop_up)):
        x_prime_down.append(x_down_in)
    x_prime_down = x_prime_down + onehop_down

    x_prime = [np.append(x_prime_up[i], x_prime_down[i]+sites) for i in range(len(x_prime_up))]
    site_double_prime = [np.intersect1d(x_prime_up[i], x_prime_down[i]) for i in range(len(x_prime))]
    double_prime = np.array([np.size(site_double_prime[i]) for i in range(len(x_prime))])

    # 跳两次的话由以下几种情况构成：up2down0， up0down2, 2*(up1down1) 下面的式子按此排序
    x_prime2_up = twohop_up.copy()
    for i in range(len(twohop_down)):
        x_prime2_up.append(x_up_in)

    x_prime2_down = []
    for i in range(len(twohop_up)):
        x_prime2_down.append(x_down_in)
    x_prime2_down = x_prime2_down + twohop_down

    len_onedown = len(onehop_down)
    for item in onehop_up:
        for _ in range(2*len_onedown):
            x_prime2_up = x_prime2_up + [item]
        x_prime2_down = x_prime2_down + onehop_down + onehop_down

    x_prime2 = [np.append(x_prime2_up[i], x_prime2_down[i]+sites) for i in range(len(x_prime2_up))]
    site_double_prime2 = [np.intersect1d(x_prime2_up[i], x_prime2_down[i]) for i in range(len(x_prime2))]
    double_prime2 = np.array([np.size(site_double_prime2[i]) for i in range(len(x_prime2))])

    psi_x_prime = np.array([np.linalg.det(phi_0[item]) for item in x_prime])
    jx_prime = np.exp(-v * double_prime)
    ux_prime = U * double_prime

    E0 = (-np.sum(jx_prime * psi_x_prime) + ux*psi_jx) / psi_jx

    psi_x_prime2 = np.array([np.linalg.det(phi_0[item]) for item in x_prime2])
    jx_prime2 = np.exp(-v * double_prime2)

    H02 = (U**2*psi_jx + np.sum(jx_prime2 * psi_x_prime2) - ux * np.sum(jx_prime * psi_x_prime) - np.sum(ux_prime * jx_prime * psi_x_prime)) / psi_jx

    return (E0, H02-E0**2)

# 由于T不改变自旋，故要分成两个子空间，并且注意边界条件！
# 顺序重要吗？ 由于认为输入的x是有序的，只要每次更改不变换位置，就不会有符号上的问题

In [74]:
energy_var(x_up, x_down)

(-10.000000000000039, -3.126388037344441e-13)